In [4]:
import pandas as pd

In [5]:
data =  pd.read_csv("channel_data.csv")
data.head()

,title,description,url
0,Rasam Vada Recipe | Tomato Rasam | Perfect Vad...,Rasam Powder: https://youtu.be/YIWBthqArCM?t=5...,https://www.youtube.com/watch?v=7LpEu6rTyHs
1,Peri Peri Chicken with Butter Rice | Spicy Hom...,Full written recipe of Chicken in Peri Peri Sa...,https://www.youtube.com/watch?v=d9_YSS1lB1w
2,Homestyle Turai Ki Sabzi | Turai Ka Bharta | त...,Full written recipe of Turai Ki Sabzi\nPrep ti...,https://www.youtube.com/watch?v=nX5vogjw_yk
3,Chicken Pepper Fry Recipe | Pepper Chicken Mas...,Full written recipe for Chicken Pepper Fry\nPr...,https://www.youtube.com/watch?v=5R6nNJC-FVE
4,Achari Paneer Recipe | Dhaba Style Paneer Grav...,Full written recipe of Achari Paneer \nPrep ti...,https://www.youtube.com/watch?v=P514UYjzZ-E


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1457 entries, 0 to 1456
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        1457 non-null   object
 1   description  1380 non-null   object
 2   url          1457 non-null   object
dtypes: object(3)
memory usage: 34.3+ KB


In [8]:
#Creating own Parser, Using LLAMA3 
import ollama

In [6]:
response = ollama.chat(model='llama3', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])

The sky appears blue because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh. Here's what happens:

When sunlight enters Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen (N2) and oxygen (O2). These molecules are much smaller than the wavelength of light itself.

The shorter wavelengths of light, like violet and blue, are scattered in all directions by these small molecules more efficiently than the longer wavelengths, like red and orange. This scattering effect is known as Rayleigh scattering.

As a result, the blue and violet light that is scattered in all directions reaches our eyes from all parts of the sky, making it appear blue to us. The color we see is actually an average of the colors that are being scattered and reaching our eyes.

Here's why other colors aren't as prominent:

* Red light has a longer wavelength than blue light, so it continues to travel in a straight line without much scattering. That's why

In [12]:
import pandas as pd
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

class RecipeInfo(BaseModel):
    recipe_name: str = Field(description="Name of the recipe short and concise like Dal bati, Red Curry, Rajma Chawal, Rasam wada")
    ingredients: List[dict] = Field(description="List of ingredients with quantities like Chickpeas: 500g , tomatoes: 2 ")
    preparation_time: int = Field(description="Preparation time in minutes like 15, 25, 40")
    cooking_time: int = Field(description="Cooking time in minutes like 10, 1hr should be 60, 45")

llm = Ollama(model="llama3")
parser = PydanticOutputParser(pydantic_object=RecipeInfo)
prompt_template = PromptTemplate(
    template=
    """Extract the following information like \n{format_instructions}\n\nfrom the below text: {text} . Be concise and short, you are a data parser for python
    so return the format as asked.
    """,
    input_variables=["text"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

def process_recipe(text):
    prompt = prompt_template.format(text=text)
    response = llm(prompt)
    parsed_response = parser.parse(response)
    return parsed_response

df = pd.read_csv('channel_data.csv')
df = df[:2]
structured_data = df['description'].apply(process_recipe)
result_df = pd.DataFrame([vars(item) for item in structured_data])
print(result_df)

# result_df.to_csv('structured_recipes.csv', index=False)

OutputParserException: Failed to parse RecipeInfo from completion {"recipe_name": "Rasam Vada", "ingredients": [{"TUR DAL | \u0924\u0942\u0930 \u0926\u093e\u0932": {"quantity": "\u00bc CUP (SOAKED)"}, "WATER | \u092a\u093e\u0928\u0940": {"quantity": "AS REQUIRED"}, "TURMERIC POWDER | \u0939\u0932\u094d\u0926\u0940 \u092a\u093e\u0909\u0921\u0930": {"quantity": "AS REQUIRED"}, "OIL | \u0924\u0947\u0932": {"quantity": "2 TBSP"}, "MUSTARD SEED | \u0930\u093e\u0908": {"quantity": "\u00bd TSP"}, "CURRY LEAVES | \u0915\u0921\u093c\u0940 \u092a\u0924\u094d\u0924\u093e": {"quantity": "12-15 NOS."}, "GREEN CHILLI | \u0939\u0930\u0940 \u092e\u093f\u0930\u094d\u091a": {"quantity": "2 NOS. (SLIT)"}, "RED CHILLI | \u0932\u093e\u0932 \u092e\u093f\u0930\u094d\u091a": {"quantity": "4 NOS."}, "GARLIC | \u0932\u0939\u0938\u0941\u0928": {"quantity": "6 CLOVES (ROUGHLY CHOPPED)"}, "ASAFOETIDA | \u0939\u0940\u0902\u0917": {"quantity": "\u00bc TSP"}, "TOMATO | \u091f\u092e\u093e\u091f\u0930": {"quantity": "2